In [ ]:
import torch
import torch.nn as nn

Designing LSTM Model Architecture

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_rate=0.2):
        super(LSTMModel, self).__init__()
        
        # LSTM layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_rate)
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout_rate)
        
        # Fully connected layer (output layer)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # x: (batch_size, seq_length, input_size)
        
        # Passing the input through LSTM layers
        lstm_out, (h_n, c_n) = self.lstm(x)
        
        # Applying dropout to the final hidden state
        out = self.dropout(lstm_out[:, -1, :])  # take output from the last time step
        
        # Passing through the fully connected output layer
        out = self.fc(out)
        
        return out

Hyperparameter Tuning

In [ ]:
input_size = 10  # Number of input features (e.g., from feature engineering)
hidden_size = 128  # Number of LSTM units
num_layers = 2  # Number of LSTM layers
output_size = 1  # Predicting market volatility
dropout_rate = 0.2  # Dropout rate

model = LSTMModel(input_size, hidden_size, num_layers, output_size, dropout_rate)

Designing CNN Model Architecture

In [ ]:
class CNNModel(nn.Module):
    def __init__(self, input_channels, num_filters, kernel_size, output_size, fc_hidden_size=128, dropout_rate=0.2):
        super(CNNModel, self).__init__()
        
        # Convolutional layer
        self.conv1 = nn.Conv2d(in_channels=input_channels, out_channels=num_filters, kernel_size=kernel_size, padding=1)
        
        # Batch normalization
        self.batchnorm1 = nn.BatchNorm2d(num_filters)
        
        # Pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Fully connected layer
        self.fc1 = nn.Linear(num_filters * (input_channels // 2) * (input_channels // 2), fc_hidden_size)
        self.dropout = nn.Dropout(dropout_rate)
        
        # Output layer
        self.fc2 = nn.Linear(fc_hidden_size, output_size)
    
    def forward(self, x):
        # x: (batch_size, input_channels, height, width)
        
        # Convolutional layer + activation
        x = self.conv1(x)
        x = torch.relu(self.batchnorm1(x))
        
        # Pooling layer
        x = self.pool(x)
        
        # Flatten the output from the convolutional layer
        x = x.view(x.size(0), -1)
        
        # Fully connected layers
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

Hyperparameter Tuning

In [ ]:
input_channels = 10  # Number of time-series features (e.g., 10 windows)
num_filters = 32  # Number of convolution filters
kernel_size = 3  # Size of convolutional filter
output_size = 1  # Predicting future price
fc_hidden_size = 128  # Hidden units in fully connected layer
dropout_rate = 0.2  # Dropout rate

cnn_model = CNNModel(input_channels, num_filters, kernel_size, output_size, fc_hidden_size, dropout_rate)